In [1]:
push!(LOAD_PATH, pwd())
using Integrator
using SpectralGrid
using CubicBSpline
using Chebyshev
using Fourier
using NumericalModels
using Parameters
using CSV
using DataFrames

┌ Info: Precompiling Integrator [top-level]
└ @ Base loading.jl:1423


In [2]:
nodes = 100
lpoints = 0
blpoints = 0
for r = 1:(nodes*3)
    lpoints += 4 + 4*r
    blpoints += 1 + 2*r
end
model = ModelParameters(
    ts = 1.0,
    integration_time = 3600.0,
    output_interval = 3600.0,
    equation_set = "LinearShallowWaterRL",
    initial_conditions = "gauss_RL_ics.csv",
    output_dir = "./linearSW_output/",
    grid_params = GridParameters(xmin = 0.0,
        xmax = 2.0e5,
        num_nodes = nodes,
        rDim = nodes*3,
        b_rDim = nodes+3,
        BCL = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R1T0, 
            "v" => CubicBSpline.R1T0),
        BCR = Dict(
            "h" => CubicBSpline.R0, 
            "u" => CubicBSpline.R0, 
            "v" => CubicBSpline.R0), 
        lDim = lpoints,
        b_lDim = blpoints,
        vars = Dict(
            "h" => 1, 
            "u" => 2, 
            "v" => 3)))

ModelParameters
  ts: Float64 1.0
  integration_time: Float64 3600.0
  output_interval: Float64 3600.0
  equation_set: String "LinearShallowWaterRL"
  initial_conditions: String "gauss_RL_ics.csv"
  output_dir: String "./linearSW_output/"
  grid_params: GridParameters


In [3]:
grid = createGrid(model.grid_params);

In [4]:
gridpoints = getGridpoints(grid)
cartesian = getCartesianGridpoints(grid)

181800×2 Matrix{Float64}:
  225.403           0.0
  159.384         159.384
    1.3802e-14    225.403
 -159.384         159.384
 -225.403           2.76039e-14
 -159.384        -159.384
   -4.14059e-14  -225.403
  159.384        -159.384
  965.926         258.819
  707.107         707.107
  258.819         965.926
 -258.819         965.926
 -707.107         707.107
    ⋮            
    1.50513e5       1.31361e5
    1.49826e5       1.32145e5
    1.49134e5       1.32925e5
    1.48438e5       1.33701e5
    1.47738e5       1.34474e5
    1.47035e5       1.35243e5
    1.46327e5       1.36009e5
    1.45615e5       1.3677e5
    1.44899e5       1.37528e5
    1.4418e5        1.38283e5
    1.43456e5       1.39033e5
    1.42729e5       1.3978e5

In [43]:
V0 = 30.0 / 50000.0
f = 5.0e-5
#V = 0.0
#U = -10.0
r_1 = 0.0
h = 0.0
v = 0.0
for i = 1:grid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    x_m = cartesian[i,1]
    y_m = cartesian[i,2]
    if (r_m < 50000.0)
        v = V0 * r_m
    else
        v = 25.0e8 * V0 / r_m
    end
    if r_m > r_1
        dhdr = ((f * v) + (v * v / r_m)) / 9.81
        h = h + (dhdr * (r_m - r_1))
        r_1 = r_m
    end
    grid.physical[i,1,1] = h
    # Gaussian blob test
    #grid.physical[i,1,1] = 100.0*exp(-((x_m-5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2))) + 100.0*exp(-((x_m+5.0e4)^2 / (2 * 1.0e4^2)) - 
    #    ((y_m)^2 / (2 * 1.0e4^2)))
    #grid.physical[i,2,1] = (U * cos(l_m)) + (V * sin(l_m))
    grid.physical[i,2,1] = 0.0
    grid.physical[i,3,1] = v
    #grid.physical[i,3,1] = -(U * sin(l_m)) + (V * cos(l_m))
end

In [44]:
gauss_ics = DataFrame(r=gridpoints[:,1],
    l=gridpoints[:,2], 
    h=grid.physical[:,1,1], 
    u=grid.physical[:,2,1], 
    v=grid.physical[:,3,1]) 

,r,l,h,u,v
,Float64,Float64,Float64,Float64,Float64
1,225.403,0.0,0.00201984,0.0,0.135242
2,225.403,0.785398,0.00201984,0.0,0.135242
3,225.403,1.5708,0.00201984,0.0,0.135242
4,225.403,2.35619,0.00201984,0.0,0.135242
5,225.403,3.14159,0.00201984,0.0,0.135242
6,225.403,3.92699,0.00201984,0.0,0.135242
7,225.403,4.71239,0.00201984,0.0,0.135242
8,225.403,5.49779,0.00201984,0.0,0.135242
9,1000.0,0.261799,0.0328142,0.0,0.6


In [45]:
gauss_ics.h

181800-element Vector{Float64}:
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.002019836696395817
   0.03281419969376366
   0.03281419969376366
   0.03281419969376366
   0.03281419969376366
   0.03281419969376366
   ⋮
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925
 103.30572853731925

In [46]:
CSV.write("gauss_SW_rankine_ics.csv",gauss_ics)

"gauss_SW_rankine_ics.csv"

In [ ]:
grid2 = initialize_model(model)

In [ ]:
ics = CSV.read("rankine_test_ic.csv", DataFrame, header=1)
vgr = ics.vgr
u = zeros(Float64,6000)
fullics = DataFrame(i=ics.i, r=ics.r, vgr=ics.vgr, u=zeros(Float64,6000), v=ics.vgr, w=zeros(Float64,6000))

In [ ]:
gp = model.grid_params
grid = createGrid(gp);

In [ ]:
V0 = 50.0 / 20000.0
for i = 1:grid.params.lDim
    r_m = gridpoints[i,1]
    l_m = gridpoints[i,2]
    if (r_m < 20000.0)
        grid.physical[i,1,1] = V0 * r_m
        grid.physical[i,2,1] = 0.0
        grid.physical[i,3,1] = V0 * r_m
        grid.physical[i,4,1] = 0.0
    else
        grid.physical[i,1,1] = 4.0e8 * V0 / r_m
        grid.physical[i,2,1] = 0.0
        grid.physical[i,3,1] = 4.0e8 * V0 / r_m
        grid.physical[i,4,1] = 0.0
    end
end

In [ ]:
i = 1
r_m = zeros(Float64,size(grid.physical[:,1,1]))
z_m = zeros(Float64,size(grid.physical[:,1,1]))
for z = 1:grid.params.zDim
    for r = 1:grid.params.rDim
        r_m[i] = grid.splines[1,1].mishPoints[r]
        z_m[i] = grid.columns[1].mishPoints[z]
        grid.physical[i,1,1] = fullics.vgr[r]
        grid.physical[i,2,1] = fullics.u[r]
        grid.physical[i,3,1] = fullics.v[r]
        grid.physical[i,4,1] = fullics.w[r]
        i += 1
    end
end

In [ ]:
rzics = DataFrame(r=r_m, z=z_m, vgr=grid.physical[:,1,1], u=grid.physical[:,2,1], v=grid.physical[:,3,1], w=grid.physical[:,4,1])

In [ ]:
CSV.write("rankine_RZ_ics.csv",rzics)